Load and merge the csv files containing the twitter data

In [1]:
import pandas as pd
import requests
from io import BytesIO
from typing import List

def load_and_merge_csv_xz_from_github(years: List[int], companies: List[str], base_url: str) -> pd.DataFrame:
    all_dataframes = []
    
    for year in years:
        for company in companies:
            filename = f"df_{year}_{company}.csv.xz"
            file_url = f"{base_url}/{filename}"
            
            response = requests.get(file_url)
            if response.status_code == 200:
                file_content = BytesIO(response.content)
                df = pd.read_csv(file_content, compression='xz')
                all_dataframes.append(df)

    merged_dataframe = pd.concat(all_dataframes, ignore_index=True)

    # Sort dataframe by date
    df_sorted = merged_dataframe.sort_values(by='post_date')

    # Remove the 'Unnamed: 0' column
    df_sorted = df_sorted.drop(columns=['Unnamed: 0'])
    
    # Drop 'year' columns that was used previously to split dataframes and save them as smaller csv files 
    df_sorted = df_sorted.drop(columns=['year'])


    # Reset the index of the dataframe and drop the old one
    df_sorted = df_sorted.reset_index(drop=True)

    return df_sorted

    
base_url = "https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data"
years = [2015, 2016, 2017, 2018, 2019]
companies = ['AAPL', 'AMZN', 'GOOGL', 'TSLA', 'GOOG', 'MSFT']
tweets_df = load_and_merge_csv_xz_from_github(years, companies, base_url)

# Optionally, you can display the first few rows of the merged DataFrame to verify it loaded correctly
# print(merged_df.head())


In [2]:
tweets_df

,ticker_symbol,company_name,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,AAPL,apple,550441509175443456,VisualStockRSRC,2015-01-01 00:00:57,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,AAPL,apple,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,0,0,0
2,AMZN,Amazon.com,550441732014223360,DozenStocks,2015-01-01 00:01:50,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,TSLA,Tesla Inc,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,TSLA,Tesla Inc,550443808606126081,aaplstocknews,2015-01-01 00:10:05,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
...,...,...,...,...,...,...,...,...,...
4336440,TSLA,Tesla Inc,1212159838882533376,ShortingIsFun,2019-12-31 23:53:21,In 2020 I may start Tweeting out positive news...,0,0,1
4336441,TSLA,Tesla Inc,1212160015332728833,Commuternyc,2019-12-31 23:54:03,Patiently Waiting for the no twitter sitter tw...,0,0,5
4336442,AAPL,apple,1212160410692046849,MoriaCrypto,2019-12-31 23:55:37,I don't discriminate. I own both $aapl and $ms...,1,0,1
4336443,MSFT,Microsoft,1212160410692046849,MoriaCrypto,2019-12-31 23:55:37,I don't discriminate. I own both $aapl and $ms...,1,0,1


In [3]:
tweets_df.columns

Index(['ticker_symbol', 'company_name', 'tweet_id', 'writer', 'post_date',
       'body', 'comment_num', 'retweet_num', 'like_num'],
      dtype='object')

In [4]:
import pandas as pd
import re

def filter_tweets_v7(df):
    df = df.copy()

    print("Initial dataset length:", len(df))

    # Normalize URLs in the tweet bodies - replace with "URL" or remove
    df['body'] = df['body'].apply(lambda text: re.sub(r'http\S+', 'URL', text))

    # Remove all duplicates based on 'tweet_id' and 'ticker_symbol' (seen as spam)
    df.drop_duplicates(subset=['tweet_id', 'ticker_symbol'], inplace=True, keep = False)
    print("After removing duplicate tweet_ids and ticker_symbols:", len(df))

    # Remove last duplicates based on 'tweet_id' and 'company_name'(will only apply to Google as they have different ticker_symbols)
    df.drop_duplicates(subset=['tweet_id', 'company_name'], inplace=True, keep = 'first')
    print("After removing duplicate tweet_ids and company_names (Google):", len(df))

    # Remove tweets with duplicate texts longer than 5 words considering the ticker symbol
    df['word_count'] = df['body'].apply(lambda text: len(text.split()))
    duplicates = df[(df['word_count'] > 5)].duplicated(subset=['body', 'ticker_symbol'], keep=False)
    df = df.loc[~df.index.isin(duplicates[duplicates].index)]
    print("After removing long duplicate texts:", len(df))

    # Apply counting functions
    df['hashtag_count'] = df['body'].apply(lambda text: sum(1 for word in text.split() if word.startswith('#')))
    df['cashtag_count'] = df['body'].apply(lambda text: sum(1 for word in text.split() if word.startswith('$')))
    df['mention_count'] = df['body'].apply(lambda text: sum(1 for word in text.split() if word.startswith('@')))

    # Filter tweets based on hashtag and cashtag counts
    df = df[(df['cashtag_count'] < 5) & (df['hashtag_count'] < 8)]
    print("After filtering by cashtags and hashtags:", len(df))

    # Ratio filters (cashtags, hashtags, mentions to words)
    df['cashtag_ratio'] = df['cashtag_count'] / df['word_count'].replace(0, 1)
    df['hashtag_ratio'] = df['hashtag_count'] / df['word_count'].replace(0, 1)
    df['mention_ratio'] = df['mention_count'] / df['word_count'].replace(0, 1)
    df = df[(df['cashtag_ratio'] <= 0.5) & (df['hashtag_ratio'] <= 0.5) & (df['mention_ratio'] <= 0.5)]
    print("After filtering by ratios:", len(df))

    # Keyword filter (defined by Wilksch, Abramova (2023) PyFin-sentiment: Towards a machine-learning-based model for deriving sentiment from financial tweets)
    keywords = ['bitcoin', 'etherium', 'btc', 'eth', 'nft', 'token', 'wallet', 'web3',
                'airdrop', 'wagmi', 'solana', 'opensea', 'cryptopunks', 'uniswap', 
                'lunar', 'hodl', 'binance', 'coinbase', 'cryptocom', 'doge']
    df['keyword_count'] = df['body'].apply(lambda text: sum(text.lower().count(kw) for kw in keywords))
    df = df[df['keyword_count'] <= 2]
    print("After filtering by keyword count:", len(df))

    return df




In [5]:
filtered_df_7 = filter_tweets_v7(tweets_df)


Initial dataset length: 4336445
After removing duplicate tweet_ids and ticker_symbols: 4336445
After removing duplicate tweet_ids and company_names (Google): 4288706
After removing long duplicate texts: 3196398
After filtering by cashtags and hashtags: 2529106
After filtering by ratios: 2508943
After filtering by keyword count: 2508222


In [6]:
# filtered_df_7[filtered_df_7['ticker_symbol'].isin(['GOOG', 'GOOGL'])].head(n=50)


In [7]:
import pandas as pd

def check_empty_bodies(df):
    # Check for empty or NaN entries in the 'body' column
    empty_bodies = df['body'].isna() | (df['body'] == '')
    
    # Count the number of empty or NaN entries
    empty_count = empty_bodies.sum()
    
    # Print the results
    if empty_count > 0:
        print(f"There are {empty_count} empty 'body' cells in the DataFrame.")
    else:
        print("There are no empty 'body' cells in the DataFrame.")

    # Optionally, you might want to return a DataFrame with the empty entries for review
    empty_entries = df[empty_bodies]
    return empty_entries


In [8]:
# Check processed data frame for missing text values
empty_entries = check_empty_bodies(filtered_df_7)
if not empty_entries.empty:
    print("Empty entries found:", empty_entries)

There are no empty 'body' cells in the DataFrame.


In [9]:
# limit magnitude of preparation as it doesn't seem to be necessary according to example in pyfin documentation
import re

def preprocess_text_column(df):
    df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    def transform_text(text):
        # Replace cashtags with 'TICKER'
        # text = re.sub(r'\$\w+', 'TICKER', text)
        
        # Replace mentions with '@user'
        text = re.sub(r'@\w+', '@user', text)
        
        # Replace all digits with '9'
        # text = re.sub(r'\d', '9', text)
        
        # Replace newlines with spaces
        text = text.replace('\n', ' ')
        
        # Convert to lowercase
        text = text.lower()

        # Remove repeated characters more than twice
        text = re.sub(r'(.)\1{2,}', r'\1', text)
        
        return text

    # Apply the transformation to the 'body' column
    df['body'] = df['body'].apply(transform_text)
    
    return df

# # To test the function, you would use:
# df = pd.DataFrame({'body': ['Helloooo!!!! How are youuuu???']})
# print(preprocess_text_column(df))

In [10]:
# Preprocess text column
preprocessed_df = preprocess_text_column(filtered_df_7)


In [11]:
# Filtering and preprocessing example
print(tweets_df["body"][3])
print(filtered_df_7["body"][3])
print(preprocessed_df["body"][3])


$GM $TSLA: Volkswagen Pushes 2014 Record Recall Tally Higher https://pic.twitter.com/WIIc1lW7hW @ProTradersNews http://growword.com/2015/01/01/0246.html… @theferrarifan
$GM $TSLA: Volkswagen Pushes 2014 Record Recall Tally Higher URL @ProTradersNews URL @theferrarifan
$gm $tsla: volkswagen pushes 2014 record recall tally higher url @user url @user


Use pyFIN to obtain sentiment scores. For now, it's commented out bc only Inga can run it without warning. We import the sentiment_df resulting from the code below.

In [12]:
# %pip install pyfin-sentiment


In [13]:
# align the scikit-learn version to pyfin to not get warnings

# FAILED. First try without version alignment

# %pip install scikit-learn==1.1.1


In [14]:
from pyfin_sentiment.model import SentimentModel
import numpy as np

# Initialize the sentiment model
# SentimentModel.download("small")  # This line is commented because it's assumed the model is already downloaded
model = SentimentModel("small")

def apply_batch_sentiment(texts, batch_size=1000):
    # Initialize an empty list to store sentiment results
    sentiments = []
    
    # Process texts in batches
    total_texts = len(texts)
    for i in range(0, total_texts, batch_size):
        batch_texts = texts[i:i + batch_size]
        batch_predictions = model.predict(batch_texts)
        sentiments.extend(batch_predictions)
        
        # Print progress
        processed = min(i + batch_size, total_texts)
        print(f"Processed {processed}/{total_texts} texts")
    
    return sentiments

# Assume preprocessed_df is your DataFrame and has already been defined
# Apply sentiment analysis in batches to the 'body' column and store the results in a new column
preprocessed_df['sentiment'] = apply_batch_sentiment(preprocessed_df['body'].tolist(), batch_size=1000)

# Now the DataFrame preprocessed_df includes a new column 'sentiment' with sentiment labels


Processed 1000/2508222 texts
Processed 2000/2508222 texts
Processed 3000/2508222 texts
Processed 4000/2508222 texts
Processed 5000/2508222 texts
Processed 6000/2508222 texts
Processed 7000/2508222 texts
Processed 8000/2508222 texts
Processed 9000/2508222 texts
Processed 10000/2508222 texts
Processed 11000/2508222 texts
Processed 12000/2508222 texts
Processed 13000/2508222 texts
Processed 14000/2508222 texts
Processed 15000/2508222 texts
Processed 16000/2508222 texts
Processed 17000/2508222 texts
Processed 18000/2508222 texts
Processed 19000/2508222 texts
Processed 20000/2508222 texts
Processed 21000/2508222 texts
Processed 22000/2508222 texts
Processed 23000/2508222 texts
Processed 24000/2508222 texts
Processed 25000/2508222 texts
Processed 26000/2508222 texts
Processed 27000/2508222 texts
Processed 28000/2508222 texts
Processed 29000/2508222 texts
Processed 30000/2508222 texts
Processed 31000/2508222 texts
Processed 32000/2508222 texts
Processed 33000/2508222 texts
Processed 34000/250

In [15]:
preprocessed_df.head(n=20)

,ticker_symbol,company_name,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num,word_count,hashtag_count,cashtag_count,mention_count,cashtag_ratio,hashtag_ratio,mention_ratio,keyword_count,sentiment
1,AAPL,apple,550441672312512512,KeralaGuy77,2015-01-01 00:01:36,insanity of today weirdo massive selling. $aap...,0,0,0,20,0,1,0,0.050000,0.000000,0.000000,0,3
3,TSLA,Tesla Inc,550442977802207232,ShowDreamCar,2015-01-01 00:06:47,$gm $tsla: volkswagen pushes 2014 record recal...,0,0,1,13,0,2,2,0.153846,0.000000,0.153846,0,1
20,AAPL,apple,550444969924653056,AppleNewsAAPL,2015-01-01 00:14:42,$aapl apple goes global with 'start something ...,0,0,1,12,0,1,1,0.083333,0.000000,0.083333,1,2
21,AAPL,apple,550444970738335744,espositooooo,2015-01-01 00:14:42,“@user: apple is being sued for falsely advert...,0,0,0,15,0,1,0,0.066667,0.000000,0.000000,0,2
22,AAPL,apple,550445066444369921,Bidnessetc,2015-01-01 00:15:05,apple filed for its own stylus patent in 2010 ...,0,0,0,11,0,1,0,0.090909,0.000000,0.000000,0,1
24,AAPL,apple,550447574285418497,btcgemini,2015-01-01 00:25:03,we searched through hundreds of charts and fou...,0,0,0,20,0,3,0,0.150000,0.000000,0.000000,0,2
31,AAPL,apple,550448085789200384,MacHashNews,2015-01-01 00:27:05,give your brain a workout with squares – chall...,0,0,0,14,1,1,0,0.071429,0.071429,0.000000,0,2
32,GOOG,Google Inc,550449439631233026,Market_Screener,2015-01-01 00:32:27,#google 'c' : chinese paper blames google over...,0,0,0,12,1,1,0,0.083333,0.083333,0.000000,0,2
33,AAPL,apple,550450683309072384,10Xtrading,2015-01-01 00:37:24,clay trader apple inc. $aapl stock chart techn...,0,0,0,11,0,1,0,0.090909,0.000000,0.000000,0,2
34,AAPL,apple,550452877466935296,TheTrendIsUp,2015-01-01 00:46:07,my biggest winner in 2014: inverse volatility ...,1,0,0,15,0,2,0,0.133333,0.000000,0.000000,0,1


In [26]:
#sentiment_df = preprocessed_df[['tweet_id', 'sentiment']]

In [27]:
#sentiment_df.to_csv('sentiment_df.csv', index=False) #saves it in the working directory

In [18]:
# Load sentiment_df

# path_sentiment_df = "https://raw.githubusercontent.com/inga-maria01/master_thesis/main/data/sentiment_df.csv.xz"

# sentiment_df = pd.read_csv(path_sentiment_df, compression='xz').drop(columns=['Unnamed: 0'])


In [19]:
# total_df = pd.merge(preprocessed_df, sentiment_df, on='tweet_id')
# print(len(total_df))


In [20]:
# total_df.head(n=20)

In [21]:
# total_df["body"][16]

In [22]:
# total_df["body"][17]

Some manual checks
- obs. 3 ('“@user: apple is being sued for falsely advertising the storage capacity of iphones: url ticker” ') is counted as neutral (2)
- obs. 9 ('my biggest winner in 9999: inverse volatility etf ticker biggest loser in 9999: apple ticker' ) is counted as positive (1)
- obs. 16 und 17 are almost equal (bezos lost bilion etc) but 16 is classified as neutral while 17 as correctly negative --> possibly because of the dots in 16, THEREFORE REMOVE DOTS

In [23]:
# try out vader
# import nltk
# nltk.download('vader_lexicon')

In [24]:
# from nltk.sentiment import SentimentIntensityAnalyzer

# # Initialize the VADER sentiment intensity analyzer
# sia = SentimentIntensityAnalyzer()

# # Assuming microsoft_tweet_details['body'] contains the text of the tweets
# # Apply the sentiment analysis
# preprocessed_df['sentiment_vader'] =preprocessed_df['body'].apply(sia.polarity_scores)

# # Extract compound scores representing a normalized, weighted composite score
# preprocessed_df['compound'] = preprocessed_df['sentiment_vader'].apply(lambda x: x['compound'])

# # Determine sentiment based on the compound score
# preprocessed_df['sentiment_type'] = preprocessed_df['compound'].apply(lambda x: 'positive' if x > 0 else ('neutral' if x == 0 else 'negative'))

# # View the distribution of sentiment types
# # print(tweet_df['sentiment_type'].value_counts())

# # View the first few rows to verify
# # print(tweet_df[['body', 'compound', 'sentiment_type']].head())


In [25]:
# preprocessed_df.head(n=20)

Weighing:
- Weight each tweet accordingly to its engagement rate consisting of number of likes, comments, retweets
    - Assign different weights to likes, comments, and retweets, e.g. likes = 1.5, comments = 2, retweets = 3 (according to a suggestion by Muñoz-Expósito et al. (2017))
- Group the weighted tweets by company for each day
    - Weigh the intensity of the sentiment score by normalized number of tweets?
- Weight the impact of each company on the index accordingly to their market cap.
    - In S&P 500 Twitter index: Index constituents are float-adjusted market capitalization (FMC) weighted, subject to a single constituent weight cap of 10%
        - FMC = Share Price × (Total Outstanding Shares × Free Float Percentage) - not relevant for nasdaq, fmc is only applied on sp500
        - The NDX's weights are adjusted quarterly
        - We probably don't need a cap as we're only including 5 big companies


Weighing:
 - Weigh by importance of company
    - Based on market value? 'market importance' -> how would that be measured?
    - Weigh based on volume traded? or weigh based on number of tweets? 
        - Could factor in the number of tweets per day normalized by the total number of tweets for that company
 - Weigh tweets by number likes/retweets/comments
 
- *Should we also weigh the importance of each day? Like if the sentiment is super extreme, but the number of tweets was very low that day (like only 50 tweets but all super positive), should we weigh it as not as telling as if there was a greater volume of tweets, so the score wouldn't be 1 it would be a bit more neutral?*